## W2-CRISPR-cBEST workflow


Base-editing with CRISPR-BEST is a DSB-free method that uses a single sgRNA to target a specific genomic location with single-nucleotide–resolution(Figure 3A) which can enable in-vivo protein engineering or as it is often used, for introducing early stop-codons10,12. It can be used to introduce targeted point mutations in the genome of Streptomyces, facilitating studies on gene function and protein engineering. To get started quickly users can download the pCRISPR-cBEST plasmid file, the genome file for S.coelicor (A3) and streptoAIM will generate your plasmids based on the genes you chose to target (Figure 3, S5). 

In [1]:
import sys
import os

# Ensure the src directory is in the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '../../'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import os
import pandas as pd
from pydna.dseqrecord import Dseqrecord
from datetime import datetime

from streptocad.sequence_loading.sequence_loading import  load_and_process_plasmid, load_and_process_genome_sequences, annotate_dseqrecord, check_and_convert_input, process_specified_gene_sequences_from_record
from streptocad.utils import polymerase_dict,ProjectDirectory, extract_metadata_to_dataframe
from streptocad.crispr.guideRNAcas3_9_12 import extract_sgRNAs, SgRNAargs
from streptocad.cloning.ssDNA_bridging import assemble_plasmids_by_ssDNA_bridging, make_ssDNA_oligos
from streptocad.crispr.crispr_best import identify_base_editing_sites, filter_sgrnas_for_base_editing, process_base_editing
from streptocad.cloning.plasmid_processing import annotate_plasmid_with_sgrnas
from streptocad.primers.primer_generation import find_best_check_primers_from_genome, create_idt_order_dataframe, primers_to_IDT

In [2]:
from pydna.dseqrecord import Dseqrecord

def find_locus_tags(record: Dseqrecord, max_tags: int = None):
    """
    Iterate through a Dseqrecord and return a list of unique locus tags found in its features,
    stopping after `max_tags` tags if specified.

    :param record: pydna.dseqrecord.Dseqrecord object
    :param max_tags: maximum number of locus tags to collect (None for no limit)
    :return: list of locus_tag strings
    """
    tags = []
    seen = set()
    count = 0

    # Ensure the record has features
    if not hasattr(record, 'features'):
        return tags

    for feature in record.features:
        if max_tags is not None and count >= max_tags:
            break

        qualifiers = getattr(feature, 'qualifiers', {})

        for tag in qualifiers.get('locus_tag', []):
            if max_tags is not None and count >= max_tags:
                break
            # only append new tags
            if tag not in seen:
                seen.add(tag)
                tags.append(tag)
                count += 1

        if max_tags is not None and count >= max_tags:
            break

    return tags


## INPUT

In [3]:
# Inputs
# 1 Add genome of choice (genbank, fasta)
path_to_genome = '../../../../data/genomes/Streptomyces_coelicolor_A3_chromosome.gb'
genome = load_and_process_genome_sequences(path_to_genome)[0]

# 2 Add plasmid 
path_to_plasmid = '../../../../data/plasmids/ pCRISPR-cBEST Sequences.gbk'
clean_plasmid = load_and_process_plasmid(path_to_plasmid)

# 3 Choose genes to knock out (list)
locus_tags = find_locus_tags(genome, 10)
genes_to_KO = locus_tags

#### Advanced settings ####
# 3 Choose polymerase and target melting temperature FOR CHECKING PRIMERS
chosen_polymerase = polymerase_dict['Phusion High-Fidelity DNA Polymerase (GC Buffer)']

melting_temperature = 65
primer_concentration = 0.4 
primer_number_increment = 1
flanking_region_number = 500

# 4 Filtering metrics for sgRNAs
gc_upper = 0.99
gc_lower = 0.01
off_target_seed = 13
off_target_upper = 10
cas_type='cas9'
number_of_sgRNAs_per_group = 1
only_stop_codons = True
flanking_region = 500

# 6 Choose overlapping sequences for our plasmid we can use the following
#As per the article **"CRISPR–Cas9, CRISPRi and CRISPR-BEST-mediated genetic manipulation in streptomycetes"** we need the following oligoes: 
#CGGTTGGTAGGATCGACGGC **-N20-** GTTTTAGAGCTAGAAATAGC
up_homology = Dseqrecord('cggttggtaggatcgacggc') # V1 homology : CGGTTGGTAGGATCGACGGC
dw_homology = Dseqrecord('tatgtcctccgagaccggcc') # V1 homology : GTTTTAGAGCTAGAAATAGC

# Computation

In [4]:
from teemi.design.fetch_sequences import read_genbank_files     
sequences = read_genbank_files(path_to_plasmid)
sequences


[SeqRecord(seq=Seq('GGCCGTCGACGACGACGCGTCCGCCTGCCTGCTTTTGGCCGGCGGGCCGGGCCG...CCA'), id='pCRISPR-cBEST(#125689)', name='pCRISPR-cBEST', description='C to T base editor for actinomycetes', dbxrefs=[])]

In [5]:
target_dict, genes_to_KO, annotation_input = check_and_convert_input(genes_to_KO)

print(annotation_input)
if annotation_input == True:
    genome = annotate_dseqrecord(genome, target_dict)



False


In [6]:
genes_to_KO

['SCO0001',
 'SCO0002',
 'SCO0003',
 'SCO0004',
 'SCO0005',
 'SCO0006',
 'SCO0007',
 'SCO0008',
 'SCO0009',
 'SCO0010']

We need to check the input genes so we can find sgRNAs. 

In [7]:

# Initialize SgRNAargs with desired parameters
args = SgRNAargs(genome, 
                genes_to_KO,
                step=['find', 'filter'],
                gc_upper = gc_upper,
                gc_lower = gc_lower,
                off_target_seed = off_target_seed,
                off_target_upper = off_target_upper,
                cas_type='cas9',
                )

sgrna_df = extract_sgRNAs(args)
sgrna_df

sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0001. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
Pam was found outside designated locus_tag: SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRNA generated were outside the designated border in SCO0002. To incorporate this extent borders. Skipping to next locus tag.
sgRN

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count
0,NC_003888.3,SCO0001,446,1,-1,7,0.55,CGG,TCCGGTAGATTCGTGGTGAC,GATTCGTGGTGAC,1
1590,NC_003888.3,SCO0005,7187,1,-1,1724,0.55,TGG,TGGCTTCATTGAAGGGCTGG,ATTGAAGGGCTGG,1
1587,NC_003888.3,SCO0005,7187,1,-1,1701,0.60,CGG,ACGTGGTCGCGGTGAATCCA,CGCGGTGAATCCA,1
1586,NC_003888.3,SCO0005,7187,1,-1,1700,0.65,GGG,CGTGGTCGCGGTGAATCCAC,GCGGTGAATCCAC,1
1585,NC_003888.3,SCO0005,7187,1,-1,1678,0.50,CGG,GATTTCGTGAAGCTGTCCGT,TGAAGCTGTCCGT,1
...,...,...,...,...,...,...,...,...,...,...,...
604,NC_003888.3,SCO0002,1252,1,1,899,0.90,CGG,GGCGGCGCTGGCCGAGGCGG,CTGGCCGAGGCGG,9
1451,NC_003888.3,SCO0005,7187,1,-1,721,0.90,CGG,GGCGCGGGCGGGTGCGGTGG,GCGGGTGCGGTGG,9
324,NC_003888.3,SCO0002,1252,1,-1,1556,0.80,CGG,CGGCGATGTCGGCGGCGTCG,GTCGGCGGCGTCG,9
386,NC_003888.3,SCO0002,1252,1,-1,2099,0.75,CGG,CGACGAGGTCGGCCAGGACG,GTCGGCCAGGACG,9


In [8]:
value_counts = sgrna_df['locus_tag'].value_counts()
value_counts

locus_tag
SCO0002    564
SCO0003    474
SCO0005    451
SCO0006    262
SCO0010    236
SCO0004    217
SCO0001    145
SCO0007    104
SCO0008     85
SCO0009     60
Name: count, dtype: int64

In [9]:
len(genome.features)

25824

In [10]:
# Load gene sequences from the genome
gene_sequences = process_specified_gene_sequences_from_record(genome, genes_to_KO)
genes_to_KO_dict = {locus_tag: gene_sequences[locus_tag] for locus_tag in genes_to_KO if locus_tag in gene_sequences}
print(genes_to_KO_dict)
# Identify and annotate base editing sites
sgrna_df_with_editing = identify_base_editing_sites(sgrna_df)

# filter out only sgRNAs that result in base-editing
filtered_sgrna_df_for_base_editing = filter_sgrnas_for_base_editing(sgrna_df_with_editing)
filtered_sgrna_df_for_base_editing

ATGACCGGTCACCACGAATCTACCGGACCAGGTACCGCGCTGAGCAGCGATTCGACGTGCCGGGTGACGCAGTATCAGACGGCGGGTGTGAACGCCCGGTTGAGACTGTTCGCGCTCCTGGAGCGCCGGGCGTGCCCGCGAGCGAGGCGGACGACCTGGTGGCCGGGGCGCAGTGCGAGGTGGTGGAGCTGGACGGCATGGCGCCGGCTTCTCGGGGTGTGCTGTGTTCGCGGACGGCTGGGACGAAGGCGTGATGGCGGTGAGCGAGGCCCTGGGGGGCATCGCGGGCCGGGACTGGCCACGGCGCGGCGCCGGTCGGGCGGGGCGACTGAGCTGGCCGTACATTGCGCGGACGTGAGGCAGCGTGAACGGGCGGACCTGGTGCGGCTGGAGGGGTTCGTCCGGGAGTCCGTGCTGCCGCGCGCCCATCCGCACACCACGGCGCGGCGCCGGGTGCTGGAGGTGCTGGGCGAGGCCGGCAGTCTGTGTACGGCGCGGACGGTGAACAGCGACGAGGACTACATCCTCTGCACGCTCGGTGTCGGGCACTACGACCCGGACGACCAACCGCCCTTCAAGGACGGAAAGCCCGGCTGGCAGAGGGCGGGTGCGTCGATCTGGAACGGCTCGGGTGCGGCCTGCATTCCGCACGCGGCCATCGAAGGACCACGGAAATAG
ATGGGGGTGTCCGCAACCGGGTCCGGTAAGACGATCACCACCGCTGCGTGCGCTCTGGAGTACTTCCCGGAGGGGCGGATCCTCGTCATGGTGCCCACGCTTGACCTGATCGTGCAGAGTGCCCAGTCCTGGCGGCGGGTCGGGCACCGGGCGCCGATGGTCGCGGTCTGCTCGGTGGACAAGGACGAGGTCCTGGAGCAGCTCGGAGTGCGCACCACCACCAACCCGATCCAGCTCGCCCTGTGGGCCGGGACCGGTCCGGTGGTCGTGTTCGCCACGTACGCCTCCCTCGTGGACCGCGAGGACCCCGCCGACGTCACT

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines
1590,NC_003888.3,SCO0005,7187,1,-1,1724,0.55,TGG,TGGCTTCATTGAAGGGCTGG,ATTGAAGGGCTGG,1,"1,0","4,7"
1587,NC_003888.3,SCO0005,7187,1,-1,1701,0.60,CGG,ACGTGGTCGCGGTGAATCCA,CGCGGTGAATCCA,1,"0,1","8,10"
1586,NC_003888.3,SCO0005,7187,1,-1,1700,0.65,GGG,CGTGGTCGCGGTGAATCCAC,GCGGTGAATCCAC,1,"0,1","7,9"
1585,NC_003888.3,SCO0005,7187,1,-1,1678,0.50,CGG,GATTTCGTGAAGCTGTCCGT,TGAAGCTGTCCGT,1,0,6
1584,NC_003888.3,SCO0005,7187,1,-1,1670,0.60,CGG,GAAGCTGTCCGTCGGTGAGT,TCCGTCGGTGAGT,1,"1,0,0","5,9,10"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,NC_003888.3,SCO0002,1252,1,1,899,0.90,CGG,GGCGGCGCTGGCCGAGGCGG,CTGGCCGAGGCGG,9,"1,1","6,8"
1451,NC_003888.3,SCO0005,7187,1,-1,721,0.90,CGG,GGCGCGGGCGGGTGCGGTGG,GCGGGTGCGGTGG,9,"1,1","5,9"
324,NC_003888.3,SCO0002,1252,1,-1,1556,0.80,CGG,CGGCGATGTCGGCGGCGTCG,GTCGGCGGCGTCG,9,"1,0","4,10"
386,NC_003888.3,SCO0002,1252,1,-1,2099,0.75,CGG,CGACGAGGTCGGCCAGGACG,GTCGGCCAGGACG,9,"0,0","4,10"


In [11]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = False, 
                                        editing_context= False)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
1590,NC_003888.3,SCO0005,7187,1,-1,1724,0.55,TGG,TGGCTTCATTGAAGGGCTGG,ATTGAAGGGCTGG,1,"1,0","4,7","E580K, A581T"
1587,NC_003888.3,SCO0005,7187,1,-1,1701,0.60,CGG,ACGTGGTCGCGGTGAATCCA,CGCGGTGAATCCA,1,"0,1","8,10","R571H, D572N"
1586,NC_003888.3,SCO0005,7187,1,-1,1700,0.65,GGG,CGTGGTCGCGGTGAATCCAC,GCGGTGAATCCAC,1,"0,1","7,9","R571H, D572N"
1585,NC_003888.3,SCO0005,7187,1,-1,1678,0.50,CGG,GATTTCGTGAAGCTGTCCGT,TGAAGCTGTCCGT,1,0,6,E565K
1584,NC_003888.3,SCO0005,7187,1,-1,1670,0.60,CGG,GAAGCTGTCCGTCGGTGAGT,TCCGTCGGTGAGT,1,"1,0,0","5,9,10",G561K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,NC_003888.3,SCO0002,1252,1,1,899,0.90,CGG,GGCGGCGCTGGCCGAGGCGG,CTGGCCGAGGCGG,9,"1,1","6,8",A302V
1451,NC_003888.3,SCO0005,7187,1,-1,721,0.90,CGG,GGCGCGGGCGGGTGCGGTGG,GCGGGTGCGGTGG,9,"1,1","5,9","A245T, R246H"
324,NC_003888.3,SCO0002,1252,1,-1,1556,0.80,CGG,CGGCGATGTCGGCGGCGTCG,GTCGGCGGCGTCG,9,"1,0","4,10","D523N, A525T"
386,NC_003888.3,SCO0002,1252,1,-1,2099,0.75,CGG,CGACGAGGTCGGCCAGGACG,GTCGGCCAGGACG,9,"0,0","4,10","D704N, V706I"


In [12]:
# Process the DataFrame to apply C-to-T mutations
mutated_sgrna_df = process_base_editing(filtered_sgrna_df_for_base_editing, 
                                        genes_to_KO_dict, 
                                        only_stop_codons = True, 
                                        editing_context= False)
mutated_sgrna_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
710,NC_003888.3,SCO0003,3869,1,-1,12,0.70,CGG,GCCACCGGCCACGGGGAAAA,GCCACGGGGAAAA,1,"0,0,1,0","5,6,9,10","W8*, V10M"
1185,NC_003888.3,SCO0004,6226,1,-1,25,0.70,CGG,CCGCCACCCGAACGGGTGAA,CCGAACGGGTGAA,1,"1,0,0,0,0","4,5,7,8,9","G13K, W14*"
1402,NC_003888.3,SCO0005,7187,1,-1,32,0.70,CGG,CGACCTGCCAGGTCTGTCCC,CCAGGTCTGTCCC,1,"0,0,1,0","4,5,8,9","W15*, V17I"
1401,NC_003888.3,SCO0005,7187,1,-1,28,0.70,AGG,CTGCCAGGTCTGTCCCCGGT,GTCTGTCCCCGGT,1,"1,0,0","4,5,10",W15*
2362,NC_003888.3,SCO0009,11617,-1,-1,36,0.65,GGG,GAGTGCTGCCAGCTGCTCCT,GCCAGCTGCTCCT,3,"1,1,0","6,9,10",Q16*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,NC_003888.3,SCO0002,1252,1,1,2364,0.75,AGG,GCGCAGTGGGTCGAGCGGGA,GGGTCGAGCGGGA,2,1,4,Q790*
415,NC_003888.3,SCO0002,1252,1,-1,2362,0.75,GGG,CCGCTCGACCCACTGCGCGA,ACCCACTGCGCGA,1,"1,0,0,0","4,6,9,10","W791*, V792I, E793K"
437,NC_003888.3,SCO0002,1252,1,1,2512,0.70,CGG,CGGAGCAGCTGAACGCGCTG,GCTGAACGCGCTG,1,"1,1","6,9",Q840*
436,NC_003888.3,SCO0002,1252,1,1,2513,0.70,GGG,GGAGCAGCTGAACGCGCTGC,CTGAACGCGCTGC,1,"1,1","5,8",Q840*


In [13]:
# Filter the DataFrame to retain only up to 5 sgRNA sequences per locus_tag
filtered_df = mutated_sgrna_df.groupby('locus_tag').head(number_of_sgRNAs_per_group)
filtered_df

,strain_name,locus_tag,gene_loc,gene_strand,sgrna_strand,sgrna_loc,gc,pam,sgrna,sgrna_seed_sequence,off_target_count,editing_context,editable_cytosines,mutations
710,NC_003888.3,SCO0003,3869,1,-1,12,0.70,CGG,GCCACCGGCCACGGGGAAAA,GCCACGGGGAAAA,1,"0,0,1,0","5,6,9,10","W8*, V10M"
1185,NC_003888.3,SCO0004,6226,1,-1,25,0.70,CGG,CCGCCACCCGAACGGGTGAA,CCGAACGGGTGAA,1,"1,0,0,0,0","4,5,7,8,9","G13K, W14*"
1402,NC_003888.3,SCO0005,7187,1,-1,32,0.70,CGG,CGACCTGCCAGGTCTGTCCC,CCAGGTCTGTCCC,1,"0,0,1,0","4,5,8,9","W15*, V17I"
2362,NC_003888.3,SCO0009,11617,-1,-1,36,0.65,GGG,GAGTGCTGCCAGCTGCTCCT,GCCAGCTGCTCCT,3,"1,1,0","6,9,10",Q16*
2297,NC_003888.3,SCO0008,11010,-1,-1,47,0.75,AGG,TTGCCAGGCGGGGCAGTCCG,GCGGGGCAGTCCG,1,"1,0,1","4,5,9","Q18*, A19V"
139,NC_003888.3,SCO0001,446,1,1,62,0.60,CGG,GGTGACGCAGTATCAGACGG,CAGTATCAGACGG,1,"0,1","6,8","T23M, Q24*"
2202,NC_003888.3,SCO0007,10554,-1,-1,94,0.70,CGG,CACACCAGCCGTTGTGGCGC,GCCGTTGTGGCGC,1,"0,0,1,0","5,6,9,10","Q34*, P35L"
2105,NC_003888.3,SCO0006,9304,1,1,101,0.70,TGG,TGATGCGCAGCCACAGGCGG,CAGCCACAGGCGG,1,"1,1","6,8","A36V, Q37*"
2377,NC_003888.3,SCO0010,12366,1,-1,96,0.65,CGG,ACCCAGGGGATATGGCCGAC,GGATATGGCCGAC,1,0,4,W38*
701,NC_003888.3,SCO0002,1252,1,1,109,0.65,TGG,TCGTGCAGAGTGCCCAGTCC,GAGTGCCCAGTCC,1,1,6,Q39*


## Output

In [14]:
# MAke oligoes
list_of_ssDNAs = make_ssDNA_oligos(filtered_df, upstream_ovh = up_homology,
                      downstream_ovh=dw_homology)


# cut plasmid
from Bio.Restriction import NcoI, NheI
linearized_plasmid = sorted(clean_plasmid.cut(NcoI,NheI ), key=lambda x: len(x), reverse=True)[0]

# assemble plasmid
sgRNA_vectors = assemble_plasmids_by_ssDNA_bridging(list_of_ssDNAs,linearized_plasmid)

In [15]:
# Constructing a meaningful name, ID, and description for the assembled plasmid using user input
targeting_info = []
for index, row in filtered_df.iterrows():
    formatted_str = f"pCRISPR-BEST_{row['locus_tag']}_p{row['sgrna_loc']}"
    targeting_info.append(formatted_str)

for i in range(len(sgRNA_vectors)):
    sgRNA_vectors[i].name = f'{targeting_info[i]}_#{i+1}'
    sgRNA_vectors[i].id = sgRNA_vectors[i].name  # Using the same value for ID as for name for simplicity
    sgRNA_vectors[i].description = f'Assembled plasmid targeting {", ".join(genes_to_KO)} for base-editing, assembled using StreptoCAD.'

# annotate plasmids
for plasmid in sgRNA_vectors: 
    annotate_plasmid_with_sgrnas(plasmid, filtered_df)

# PRINT
print_plasmids = False
if print_plasmids: 
    for vector in sgRNA_vectors: 
        vector.write(f"../../data/plasmids/sgRNA_plasmids_pCRISPR_cBEST/{vector.id}.gb")

In [16]:
integration_names = filtered_df.apply(lambda row: f"sgRNA_{row['locus_tag']}({row['sgrna_loc']})", axis=1).tolist()
plasmid_metadata_df = extract_metadata_to_dataframe(sgRNA_vectors,
                                                    clean_plasmid,
                                                    integration_names)

plasmid_metadata_df

,plasmid_name,date,original_plasmid,integration,size
0,pCRISPR-BEST_SCO0003_p12_#1,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0003(12),11817
1,pCRISPR-BEST_SCO0004_p25_#2,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0004(25),11817
2,pCRISPR-BEST_SCO0005_p32_#3,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0005(32),11817
3,pCRISPR-BEST_SCO0009_p36_#4,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0009(36),11817
4,pCRISPR-BEST_SCO0008_p47_#5,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0008(47),11817
5,pCRISPR-BEST_SCO0001_p62_#6,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0001(62),11817
6,pCRISPR-BEST_SCO0007_p94_#7,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0007(94),11817
7,pCRISPR-BEST_SCO0006_p101_#8,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0006(101),11817
8,pCRISPR-BEST_SCO0010_p96_#9,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0010(96),11817
9,pCRISPR-BEST_SCO0002_p109_#10,2025-06-10,pCRISPR-cBEST(#125689),sgRNA_SCO0002(109),11817


# IDT primers

In [17]:
full_idt = primers_to_IDT(list_of_ssDNAs)
full_idt

,Name,Sequence,Concentration,Purification
0,SCO0003_loc_12,cggttggtaggatcgacggcGCCACCGGCCACGGGGAAAAtatgtc...,25nm,STD
1,SCO0004_loc_25,cggttggtaggatcgacggcCCGCCACCCGAACGGGTGAAtatgtc...,25nm,STD
2,SCO0005_loc_32,cggttggtaggatcgacggcCGACCTGCCAGGTCTGTCCCtatgtc...,25nm,STD
3,SCO0009_loc_36,cggttggtaggatcgacggcGAGTGCTGCCAGCTGCTCCTtatgtc...,25nm,STD
4,SCO0008_loc_47,cggttggtaggatcgacggcTTGCCAGGCGGGGCAGTCCGtatgtc...,25nm,STD
5,SCO0001_loc_62,cggttggtaggatcgacggcGGTGACGCAGTATCAGACGGtatgtc...,25nm,STD
6,SCO0007_loc_94,cggttggtaggatcgacggcCACACCAGCCGTTGTGGCGCtatgtc...,25nm,STD
7,SCO0006_loc_101,cggttggtaggatcgacggcTGATGCGCAGCCACAGGCGGtatgtc...,25nm,STD
8,SCO0010_loc_96,cggttggtaggatcgacggcACCCAGGGGATATGGCCGACtatgtc...,25nm,STD
9,SCO0002_loc_109,cggttggtaggatcgacggcTCGTGCAGAGTGCCCAGTCCtatgtc...,25nm,STD


## Folder with all the generated I/O

In [18]:
input_files = [
    {"name": "input_genome.gb", "content": genome},
    {"name": "input_plasmid.gb", "content": clean_plasmid}
]

output_files = [
    {"name": "cBEST_w_sgRNAs.gb", "content": sgRNA_vectors}, # LIST OF Dseqrecords
    {"name": "full_idt.csv", "content": full_idt},
    {"name": "mutated_sgrna_df.csv", "content": mutated_sgrna_df},
    {"name": "filtered_df.csv", "content": filtered_df},
    {"name": "plasmid_metadata_df.csv", "content": plasmid_metadata_df},

]

input_values = {
    "genes_to_knockout": genes_to_KO,
    "polymerase_settings": {
        "chosen_polymerase": chosen_polymerase,
        "melting_temperature": melting_temperature,
        "primer_concentration": primer_concentration,
        "primer_number_increment": primer_number_increment,
        "flanking_region_number": flanking_region_number
    },
    "filtering_metrics": {
        "gc_upper": gc_upper,
        "gc_lower": gc_lower,
        "off_target_seed": off_target_seed,
        "off_target_upper": off_target_upper,
        "cas_type": cas_type,
        "number_of_sgRNAs_per_group": number_of_sgRNAs_per_group
    },
    "overlapping_sequences": {
        "up_homology": str(up_homology),
        "dw_homology": str(dw_homology)
    }
}


# Paths to Markdown files
markdown_file_paths = [
    "../../protocols/conjugation_protcol.md",
    "../../protocols/single_target_crispr_plasmid_protcol.md"

]

# Data and time
timestamp = datetime.utcnow().isoformat()

# Create project directory structure
project_directory = ProjectDirectory(
    project_name=f"CRISPR_cBEST_workflow_{timestamp}",
    input_files=input_files,
    output_files=output_files,
    input_values=input_values,
    markdown_file_paths=markdown_file_paths
)


# DO You want to save the folder? 
save_zip_folder = False 

if save_zip_folder: 
    # Generate the project directory structure and get the zip content
    zip_content = project_directory.create_directory_structure(create_directories=False)

    # Save the zip file to disk (optional)
    with open("project_structure.zip", "wb") as f:
        f.write(zip_content)